In [ ]:
import pandas as pd 
import os

os.chdir("/Volumes/SAMSUNG/trpred")

# Get all trp users from submissions
users_submissions = pd.read_csv("data/processed/submissions/TheRedPill-allsubmissions-2020-05-16-metadata.csv", usecols = ["id","author"])

# Get submission ids to search for author flair by submission
ls_subids = list(users_submissions.id)


In [ ]:
test_id = "993wvk"
test_id

In [ ]:
# https://api.pushshift.io/reddit/submission/search/?ids=9c7knc
import requests 

url = "https://api.pushshift.io/reddit/submission/search/?ids="
url = url + test_id

url


In [ ]:
# initializing this here so it doesn't restart every time i rerun it
trpuser_flairs = {}

In [ ]:
import numpy as np 
import requests 
import time 
import pandas as pd
import json
import os 

os.chdir("/Volumes/SAMSUNG/trpred")

counter = 0
filename = "data/info/20201004_trp_use_flairs.json"

for i in ls_subids:
    # set url to search by submission id
    url = "https://api.pushshift.io/reddit/submission/search/?ids="
    url = url + i # concat id from list
    r = requests.get(url) # make request
    
    author = users_submissions.loc[users_submissions['id'] == i]['author']
    author = author.values[0]
    
    try:
        flair = r.json()['data'][0]['author_flair_text'] # get flair if it exists
    except:
        flair = np.nan # otherwise, no flair
    
    trpuser_flairs[author] = flair
    
    print(i)
    counter += 1 
    print("og start +:", counter)
    
    time.sleep(1)

    if len(trpuser_flairs.keys()) % 1000 == 0:
        try:
            with open(filename, 'wb') as outfile:
                json.dump(trpuser_flairs, outfile)
        except:
            continue

    
    
    
    
    

In [ ]:
len(trpuser_flairs.keys())
len(ls_subids)

In [ ]:
# Fix up the DF
df_flairs = pd.DataFrame.from_dict(subred_descriptions, orient = "index", columns = ["flair"])
df_flairs = df_flairs.reset_index()
df_flairs = df_flairs.rename(columns = {"index":"user"})


In [ ]:
# Save to csv
os.chdir("/Volumes/SAMSUNG/trpred")

df_flairs.to_csv("info/trp_use_flairs.csv", mode = "w")

In [ ]:
import pandas as pd 
import os

os.chdir("/Volumes/SAMSUNG/trpred")

# Get all trp users from comments
users_comments = pd.read_csv("data/processed/comments/TheRedPill-comments-2020-06-01.csv", usecols = ["author", "author_flair_text"])

## NEED TO REPROCESS R/TRP COMMENTS BC OF STUPID ICLOUD



In [ ]:
users_comments

In [ ]:
import pandas as pd
import re
import utils
import datetime as dt
import csv
import os
import b_clean_data as clean
from os import listdir
from os.path import isfile, join
from fsplit.filesplit import FileSplit
import glob
from os import listdir
from os.path import isfile, join

In [ ]:
comment_files[4]

In [ ]:

import pandas as pd
import re
import utils
import datetime as dt
import csv
import os
from os import listdir
from os.path import isfile, join
from fsplit.filesplit import FileSplit
import glob
from os import listdir
from os.path import isfile, join

def clean_comments(raw_comments):
    """Cleans comments found in the folder passed in."""

    new_folder = utils.create_folder(raw_comments)

    if new_folder == None:
        print("No directory created")
        new_folder = "data/raw/comments/" + utils.get_filename(raw_comments)

    # Get name for processed file
    print("raw_comments = " + raw_comments)

    
    regex = r"([^\/]+)(?=.allcomments)"
    matches = re.search(regex, raw_comments)
    new_file = matches.group(1)

    # Comment files
    files = [f for f in listdir(new_folder) if isfile(join(new_folder, f))]
    print(*files, sep = "\n")
    
    # Create list of columns to keep
    keep_cols = ['id', 'created_utc','author',\
                  'author_flair_text', 'score', 'parent_id',\
                  'subreddit']
    keep_cols_text = ['id', 'created_utc', 'parent_id', 'body']

    # Create file name
    processedfile_csv = "data/processed/comments/" + new_file + \
        "-metadata" +  ".csv"

    processed_textfile_csv = "data/processed/comments/" + new_file + \
        "-text" + ".csv"

    counter = 0

    # Read in json file
    for i in files:
        print(i)
        counter += 1

        df_keep = pd.DataFrame()
        df_keep_text = pd.DataFrame()

        file_path = new_folder + "/" + i

        try:
            data = pd.read_json(file_path)

        # ValueError: Trailing data thrown if file is pretty indented
        except ValueError:
            data = pd.read_json(file_path, lines = True)


        try:
            df_keep = df_keep.append(data[keep_cols])
        except KeyError:
            keep_cols = ['id', 'created_utc', 'author', 'title',\
                        'score', 'num_comments', 'subreddit']
            df_keep = df_keep.append(data[keep_cols])


        try:
            df_keep_text = df_keep_text.append(data[keep_cols_text])
        except KeyError:
            keep_cols_text = ['id', 'created_utc', 'author']
            df_keep_text = df_keep_text.append(data[keep_cols_text])

        # Make sure there's at least 1 observation
        observations = len(df_keep.index)

        # Change date format
        ## For metadata
        if observations == 0:
            print("No comments found in " + i)
            continue

        else:
            df_keep['datetime_dv'] = pd.to_datetime(df_keep['created_utc'], unit = 's')# dv = derived
            df_keep['date_dv'] = df_keep['datetime_dv'].dt.date

            # For text
            df_keep_text['datetime_dv'] = pd.to_datetime(df_keep_text['created_utc'], unit = 's')# dv = derived
            df_keep_text['date_dv'] = df_keep_text['datetime_dv'].dt.date


        ##### Delimit by date #####
        start = dt.datetime.fromtimestamp(utils.get_startdate()).date()
        end = dt.datetime.fromtimestamp(utils.get_enddate()).date()

        # Create mask of time slot
        mask = (df_keep['date_dv'] >= start) & (df_keep['date_dv'] <= end) # inclusive on either end
        mask_text = (df_keep_text['date_dv'] >= start) & (df_keep_text['date_dv'] <= end)

        # Only keep data within date frame
        df_keep = df_keep.loc[mask]
        df_keep_text = df_keep_text.loc[mask_text]
        ############################

        # Save to csv
        if counter == 1:
            df_keep.to_csv(processedfile_csv, mode = "w")
            df_keep_text.to_csv(processed_textfile_csv, mode = "w")

        else:
            df_keep.to_csv(processedfile_csv, mode = "a", header = False)
            df_keep_text.to_csv(processed_textfile_csv, mode = "a", header = False)







In [ ]:
## Clean comments and submissions
os.chdir("/Volumes/SAMSUNG/trpred")# Get comment folders

# comment_folders = [x[0] for x in os.walk("data/raw/comments")]
# comment_folders = comment_folders[1:] # item at index 0 is parent folder so exclude

comment_files = glob.glob("data/raw/comments/*.json")


# comment_files = ["data/raw/comments/Braincels-allcomments-2020-10-02.json",
# "data/raw/comments/exredpill-allcomments-2020-10-03.json",
# "data/raw/comments/fPUA-allcomments-2020-10-03.json",
# "data/raw/comments/IncelTears-allcomments-2020-10-03.json",
# "data/raw/comments/KotakuInAction-allcomments-2020-10-06.json",
# "data/raw/comments/masculism-allcomments-2020-10-03.json",
# "data/raw/comments/MensRights-allcomments-2020-10-02.json",
# "data/raw/comments/MGTOW-allcomments-2020-10-03.json",
# "data/raw/comments/PurplePillDebate-allcomments-2020-10-03.json",
# "data/raw/comments/TheRedPill-allcomments-2020-10-01.json"]


for i in comment_files:
    print(i)
    clean_comments(i)
    print(i + " end")
    
    
### NEW FILE ISNT BEING CREATED -- CHECK WHY

In [ ]:
comment_files[4:]

In [ ]:
test = "data/raw/comments/KotakuInAction_allcomments_10-06.json"
regex = r"([^\/]+)(?=.all)"
matches = re.search(regex, test)
new_file = matches.group(1)

In [ ]:
new_file

In [ ]:
test_new = dt.datetime.fromtimestamp(test).date()
type(test_new)

In [73]:
import pandas as pd 
import datetime as dt
import utils

def clean_submissions(raw_submissions):
    """Cleans the file passed in.
    Saves files to processed/submissions folder within trpred."""

    # Get name for processed file
    regex = r"([^\/]+)(?=.allsubmissions)"
    matches = re.search(regex, raw_submissions)
    new_file = matches.group(1)

    # Create list of columns to keep
    keep_cols = ['id', 'created_utc', 'author', 'title',\
                  'score', 'num_comments', 'subreddit', 'link_flair_text']

    keep_cols_text = ['id', 'created_utc', 'author', 'selftext']

    # Create file name
    processedfile_csv = "data/processed/submissions/" + new_file + \
        "-metadata" +  ".csv"

    processed_textfile_csv = "data/processed/submissions/" + new_file + \
        "-text" + ".csv"

    # Create empty data frame
    df_keep = pd.DataFrame()
    df_keep_text = pd.DataFrame()

    # Read in json file
    try:
        data = pd.read_json(raw_submissions)

    # ValueError: Trailing data thrown if file is pretty indented
    except ValueError:
        data = pd.read_json(raw_submissions, lines = True)

    try:
        df_keep = df_keep.append(data[keep_cols])
    except KeyError:
        keep_cols = ['id', 'created_utc', 'author', 'title',\
                    'score', 'num_comments', 'subreddit']
        df_keep = df_keep.append(data[keep_cols])

    try:
        df_keep_text = df_keep_text.append(data[keep_cols_text])
    except KeyError:
        keep_cols_text = ['id', 'created_utc', 'author']
        df_keep_text = df_keep_text.append(data[keep_cols_text])


    # Change date format
    ## For metadata
    df_keep['datetime_dv'] = pd.to_datetime(df_keep['created_utc'], unit = 's')# dv = derived
    df_keep['date_dv'] = df_keep['datetime_dv'].dt.date

    # For text
    df_keep_text['datetime_dv'] = pd.to_datetime(df_keep_text['created_utc'], unit = 's')# dv = derived
    df_keep_text['date_dv'] = df_keep_text['datetime_dv'].dt.date


    ##### Delimit by date #####
    start = dt.datetime.fromtimestamp(utils.get_startdate()).date()
    end = dt.datetime.fromtimestamp(utils.get_enddate()).date()

    # Create mask of time slot
    mask = (df_keep['date_dv'] >= start) & (df_keep['date_dv'] <= end) # inclusive on either end
    mask_text = (df_keep_text['date_dv'] >= start) & (df_keep_text['date_dv'] <= end)

    # Only keep data within date frame
    df_keep = df_keep.loc[mask]
    df_keep_text = df_keep_text.loc[mask_text]
    ############################


    # Save to json
    df_keep_text.to_csv(processed_textfile_csv, mode = "w")
    df_keep.to_csv(processedfile_csv, mode = "w") # mode= w will overwrite previous file


    data = [] # force empty

In [ ]:
import glob 
os.chdir("/Volumes/SAMSUNG/trpred")


processed_subs = glob.glob("data/processed/submissions/*.csv")

all_subs = glob.glob("data/raw/submissions/*.json")


In [42]:
import re 
import os

clean_processed = []

for i in processed_subs:
    regex = r"([^\/]+)(?=\-)" # set regex to get file names
    matches = re.search(regex, i)
    filename = matches.group()
    clean_processed.append(filename)
 
clean_processed = set(clean_processed)

all_clean = []
for i in all_subs:
    regex = r"([^\/]+)(?=\-all)" # set regex to get file names
    matches = re.search(regex, i)
    filename = matches.group()
    all_clean.append(filename)


all_clean = set(all_clean)


In [ ]:
all_subs

In [74]:
remaining = [x for x in all_clean if x not in clean_processed]

remaining_files = ['data/raw/submissions/theredpillright-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/askseddit-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/thankTRP-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/redpillfatherhood-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/redpillbooks-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/mgtowbooks-allsubmissions-2020-08-19.json',
                   'data/raw/submissions/mensrightslaw-allsubmissions-2020-08-15.json',
                   'data/raw/submissions/seduction-allsubmissions-2020-08-23.json',
                   'data/raw/submissions/askTRP-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/marriedredpill-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/exredpill-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/becomeaman-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/pua-allsubmissions-2020-08-19.json',
                   'data/raw/submissions/theRedPillLeft-allsubmissions-2020-08-13.json',
                   'data/raw/submissions/masculism-allsubmissions-2020-08-14.json',
                   'data/raw/submissions/badwomensanatomy-allsubmissions-2020-08-13.json']

remaining_files[-1]

'data/raw/submissions/badwomensanatomy-allsubmissions-2020-08-13.json'

In [76]:
os.chdir("/Volumes/SAMSUNG/trpred")

for i in remaining_files[-1:]:
    clean_submissions(i)
    print(i + " done")


data/raw/submissions/badwomensanatomy-allsubmissions-2020-08-13.json done


In [60]:
# missing one subreddit
os.chdir("/Volumes/SAMSUNG/trpred")

cleaned = glob.glob("data/processed/submissions/*.csv")

cleaned_ls = []
for i in cleaned:
    regex = r"([^\/]+)(?=\-)" # set regex to get file names
    matches = re.search(regex, i)
    filename = matches.group()
    cleaned_ls.append(filename) 

cleaned_ls = set(cleaned_ls)

100sets
100sets
AskFeminists
AskFeminists
BlackPillScience
BlackPillScience
Braincels
Braincels
Egalitarianism
Egalitarianism
EthnicRedPill
EthnicRedPill
FeMRADebates
FeMRADebates
GEOTRP
GEOTRP
IncelTears
IncelTears
IncelsInAction
IncelsInAction
IncelsWithoutHate
IncelsWithoutHate
KotakuInAction
KotakuInAction
LeftWingMaleAdvocates
LeftWingMaleAdvocates
MGTOW
MGTOW
MGTOW2
MGTOW2
MGTOWmusic
MGTOWmusic
MRActivism
MRActivism
MRAmemes
MRAmemes
MensRants
MensRants
MensRights
MensRights
MensRightsMeta
MensRightsMeta
NOMAAM
NOMAAM
PurplePillDebate
PurplePillDebate
RedPillLit
RedPillLit
RedPillNonMonogamy
RedPillNonMonogamy
RedPillParenting
RedPillParenting
RedPillReadingGroup
RedPillReadingGroup
RedPillScience
RedPillScience
RedPillWives
RedPillWives
RedPillWomen
RedPillWomen
RedPillWorkplace
RedPillWorkplace
TRPOffTopic
TRPOffTopic
TRPmemes
TRPmemes
TheRedPill
TheRedPill
Trufemcels
Trufemcels
TumblrInAction
TumblrInAction
WhereAreAllTheGoodMen
WhereAreAllTheGoodMen
altTRP
altTRP
askTRP
askTR

In [65]:
cleaned_ls

missing_one = [x for x in all_clean if x not in cleaned_ls]

In [66]:
missing_one

['badwomensanatomy']